In [7]:
import argparse
import matplotlib 
matplotlib.use('Agg')
import math
import datetime
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import re

%matplotlib widget


# Update rate

In [2]:
f = open("sbtunnel.log", "r")

changes = []
raw_lines = []
net = 0
total_changes = 0
for l in f.readlines():
    if re.match(r".*FETCH Response.*", l):
        #print(re.match(r".*FETCH.*", l))
        
        date_time_str = re.search(r"(.*) FETCH.*", l).group(1)
        date_time_obj = datetime.datetime.strptime(date_time_str, '%Y/%m/%d %H:%M:%S')
        ts = date_time_obj.timestamp()
        
        added = int(re.search(r"RiceHashes added: (\d+)", l).group(1))
        removed = 0
        if re.match(r".*FULL_UPDATE.*", l) == None:
            removed = int(re.search(r"RiceIndices removed: (\d+)", l).group(1))
            
        delta = added+removed
        net += added - removed
        total_changes += delta
        changes += [[ts, added, removed, delta, net, total_changes]]
        raw_lines += [l]

In [10]:
changes = np.array(changes)

plt.figure()
plt.title("Total number of partial hashes")
plt.plot(changes[:,0], changes[:,4])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
plt.figure()
plt.plot((changes[1:,0]-changes[0,0])/3600, changes[1:,5]-changes[0,5])
plt.title("Cumulative number of changes")
plt.xlabel("Time [hours]")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
coef = np.polyfit(changes[:,0], changes[:,5],1)
print(f"Changes per hour: {coef[0]*3600}")

coef = np.polyfit(changes[1:,0]-changes[0,0], np.cumsum(changes[1:,1]),1)
print(f"Additions per hour: {coef[0]*3600}")

coef = np.polyfit(changes[1:,0]-changes[0,0], np.cumsum(changes[1:,2]),1)
print(f"Deletions per hour: {coef[0]*3600}")

Changes per hour: 702.6050195508681
Additions per hour: 465.7005348095512
Deletions per hour: 235.67649111020455


# Query rate

In [18]:
f = open("sbtunnel.log", "r")

queries = []
for l in f.readlines():
    if re.match(r".*FIND Request.*", l):
        date_time_str = re.search(r"(.*) FIND.*", l).group(1)
        date_time_obj = datetime.datetime.strptime(date_time_str, '%Y/%m/%d %H:%M:%S')
        ts = date_time_obj.timestamp()
        
        q = int(re.search(r".*(\d+) prefixes", l).group(1))
            
        queries += [[ts, q]]

In [26]:
queries = np.array(queries)
plt.figure()
plt.plot((queries[:,0]-queries[0,0])/3600)
plt.title("Query rate")
plt.xlabel("# queries")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [30]:
window_start = queries[0,0]
window_end = queries[1,0]
window_q = 0
windows = []
for q in queries:
    if (q[0] - window_end) > 30*60:
        windows += [[int((window_end-window_start)/60), window_q]]
        window_start = q[0]
        window_end = window_start
        window_q = 1
    else:
        window_end = q[0]
        window_q += 1
print(windows)

[[143, 33], [63, 16], [0, 1], [2, 3], [45, 3], [0, 1], [172, 46], [31, 12], [23, 5], [21, 3], [58, 22], [19, 3], [50, 14], [106, 24], [61, 30], [39, 6], [13, 6], [45, 6], [49, 9], [117, 23], [33, 9], [27, 4], [13, 8], [30, 7], [106, 20], [0, 1], [0, 1]]


In [32]:
windows = np.array(windows)
windows[:,0]/windows[:,1]

array([ 4.33333333,  3.9375    ,  0.        ,  0.66666667, 15.        ,
        0.        ,  3.73913043,  2.58333333,  4.6       ,  7.        ,
        2.63636364,  6.33333333,  3.57142857,  4.41666667,  2.03333333,
        6.5       ,  2.16666667,  7.5       ,  5.44444444,  5.08695652,
        3.66666667,  6.75      ,  1.625     ,  4.28571429,  5.3       ,
        0.        ,  0.        ])